# Abstract Document Summarization with Langchain using Mistral Large on Bedrock


## Overview
This notebook is meant to demonstrate using the [Mistral models](https://docs.mistral.ai/deployment/cloud/aws/) on Amazon Bedrock for Abstract document summarization tasks. Although all the Mistral models have relatively large context window sizes, when working with multiple large documents, there are several challenges that can arise. One of the main challenges is that the input text might exceed the model's context length. This limitation can lead to incomplete or inaccurate responses, as the model may not have access to all the relevant information within the document. Another challenge is that language models can sometimes hallucinate or generate factually incorrect responses when dealing with very long documents. This can happen because the model may lose track of the overall context or make incorrect inferences based on partial information. Additionally, processing large documents can lead to out-of-memory errors, especially on resource-constrained systems or when working with large language models that have high memory requirements.

To address these challenges, this notebook will go through various summarization strategies that will use [LangChain](https://python.langchain.com/docs/get_started/introduction.html), a popular framework for developing applications powered by large language models (LLMs).


---
## Mistral Model Selection

Today, there are three Mistral models available on Amazon Bedrock:

### 1. Mistral 7B Instruct

- **Description:** A 7B dense Transformer model, fast-deployed and easily customizable. Small yet powerful for a variety of use cases.
- **Max Tokens:** 8,196
- **Context Window:** 32K
- **Languages:** English
- **Supported Use Cases:** Text summarization, structuration, question answering, and code completion

### 2. Mixtral 8X7B Instruct

- **Description:** A 7B sparse Mixture-of-Experts model with stronger capabilities than Mistral 7B. Utilizes 12B active parameters out of 45B total.
- **Max Tokens:** 4,096
- **Context Window:** 32K
- **Languages:** English, French, German, Spanish, Italian
- **Supported Use Cases:** Text summarization, structuration, question answering, and code completion

### 3. Mistral Large

- **Description:** A cutting-edge text generation model with top-tier reasoning capabilities. It can be used for complex multilingual reasoning tasks, including text understanding, transformation, and code generation.
- **Max Tokens:** 8,196
- **Context Window:** 32K
- **Languages:** English, French, German, Spanish, Italian
- **Supported Use Cases:** Synthetic Text Generation, Code Generation, RAG, or Agents

### Performance and Cost Trade-offs

The table below compares the model performance on the Massive Multitask Language Understanding (MMLU) benchmark and their on-demand pricing on Amazon Bedrock.

| Model           | MMLU Score | Price per 1,000 Input Tokens | Price per 1,000 Output Tokens |
|-----------------|------------|------------------------------|-------------------------------|
| Mistral 7B Instruct | 62.5%      | \$0.00015                    | \$0.0002                      |
| Mixtral 8x7B Instruct | 70.6%      | \$0.00045                    | \$0.0007                      |
| Mistral Large | 81.2%      | \$0.008                   | \$0.024                     |

For more information, refer to the following links:

1. [Mistral Model Selection Guide](https://docs.mistral.ai/guides/model-selection/)
2. [Amazon Bedrock Pricing Page](https://aws.amazon.com/bedrock/pricing/)

---

### Local Setup (Optional)

For a local server, follow these steps to execute this jupyter notebook:

1. **Configure AWS CLI**: Configure [AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html) with your AWS credentials. Run `aws configure` and enter your AWS Access Key ID, AWS Secret Access Key, AWS Region, and default output format.

2. **Install required libraries**: Install the necessary Python libraries for working with SageMaker, such as [sagemaker](https://github.com/aws/sagemaker-python-sdk/), [boto3](https://github.com/boto/boto3), and others. You can use a Python environment manager like [conda](https://docs.conda.io/en/latest/) or [virtualenv](https://virtualenv.pypa.io/en/latest/) to manage your Python packages in your preferred IDE (e.g. [Visual Studio Code](https://code.visualstudio.com/)).

3. **Create an IAM role for SageMaker**: Create an AWS Identity and Access Management (IAM) role that grants your user [SageMaker permissions](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html). 

By following these steps, you can set up a local Jupyter Notebook environment capable of deploying machine learning models on Amazon SageMaker using the appropriate IAM role for granting the necessary permissions.

## Requirements

---
1. Create an Amazon SageMaker Notebook Instance - [Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/gs-setup-working-env.html)
    - For Notebook Instance type, choose ml.t3.medium.
2. For Select Kernel, choose [conda_pytorch_p310](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-prepare.html).
3. Install the required packages.

---

Before we start building the agentic workflow, we'll first install some libraries:

+ AWS Python SDKs [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) to be able to submit API calls to [Amazon Bedrock](https://aws.amazon.com/bedrock/).
+ [LangChain](https://python.langchain.com/v0.1/docs/get_started/introduction/) is a framework that provides off the shelf components to make it easier to build applications with large language models. It is supported in multiple programming languages, such as Python, JavaScript, Java and Go. 

---

In [1]:
%%writefile requirements.txt
langchain==0.1.14
boto3==1.34.58
botocore==1.34.101
sqlalchemy==2.0.29
pypdf==4.1.0
langchain-aws==0.1.6
transformers

Overwriting requirements.txt


In [2]:
!pip install -U -r requirements.txt --quiet

#### Restart the kernel with the updated packages that are installed through the dependencies above

---


## Initiate the Bedrock Client

Import the necessary libraries, along with langchain for bedrock model selection

In [3]:
import boto3
from boto3 import client
from botocore.config import Config
import json
from langchain_aws import ChatBedrock
from langchain.chains import ConversationChain
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
import numpy as np
from pypdf import PdfReader
from urllib.request import urlretrieve

In [4]:
config = Config(read_timeout=2000)

bedrock = boto3.client(service_name='bedrock-runtime', 
                       region_name='us-west-2',
                       config=config)

<div class="alert alert-block alert-warning"> 

<b>NOTE:</b> Ensure that you have access to the Mistral model you wish to use through Bedrock.
</div>

## Configure LangChain with Boto3

---


With LangChain, you can access Bedrock once you pass the boto3 session information to LangChain. Below, we also specify Mistral Large in `model_id` and pass Mistral's inference parameters as desired in `model_kwargs`.



---
### Supported parameters

The Mistral AI models have the following inference parameters.


```
{
    "prompt": string,
    "max_tokens" : int,
    "stop" : [string],    
    "temperature": float,
    "top_p": float,
    "top_k": int
}
```

The Mistral AI models have the following inference parameters:

- **Temperature** - Tunes the degree of randomness in generation. Lower temperatures mean less random generations.
- **Top P** - If set to float less than 1, only the smallest set of most probable tokens with probabilities that add up to top_p or higher are kept for generation.
- **Top K** - Can be used to reduce repetitiveness of generated tokens. The higher the value, the stronger a penalty is applied to previously present tokens, proportional to how many times they have already appeared in the prompt or prior generation.
- **Maximum Length** - Maximum number of tokens to generate. Responses are not guaranteed to fill up to the maximum desired length.
- **Stop sequences** - Up to four sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.

---


In [5]:
#Set the desired mistral model as the default model
instruct_mistral7b_id = "mistral.mistral-7b-instruct-v0:2"
instruct_mixtral8x7b_id = "mistral.mixtral-8x7b-instruct-v0:1"
mistral_large_2402_id = "mistral.mistral-large-2402-v1:0"

DEFAULT_MODEL = mistral_large_2402_id

In [6]:
llm = ChatBedrock(
    model_id=DEFAULT_MODEL,
    model_kwargs={
        "max_tokens": 8192,  ## MAXIMUM NUMBER OF TOKENS for Mistral Large
        "temperature": 0.5,
        "top_p": 1
    },
    client=bedrock,
)

In [7]:
#Initialize conversation chain with Mistral Large on Bedrock
conversation = ConversationChain(
    llm=llm, verbose=False, memory=ConversationBufferMemory()
)

conversation.predict(input="Hi there!")

" Hello! I'm delighted to chat with you. How can I assist you today or is there a particular topic you'd like to discuss? Remember, I'm here to provide information, answer questions, or simply engage in a friendly conversation."

---

## Document Processing Step

In this example, to demonstrate summarization, we will be using two documents that are both whitepapers from AWS. 

> The first document is a [whitepaper](https://docs.aws.amazon.com/whitepapers/latest/architecting-hipaa-security-and-compliance-on-aws/architecting-hipaa-security-and-compliance-on-aws.pdf) on architecting HIIPA compliant workloads on AWS.

> The second document is a [whitepaper](https://docs.aws.amazon.com/whitepapers/latest/containers-on-aws/containers-on-aws.pdf) about containers on AWS. 

Let's first download these files to build our document store.

In [8]:
!mkdir -p ./data

urls = [
    'https://docs.aws.amazon.com/whitepapers/latest/architecting-hipaa-security-and-compliance-on-aws/architecting-hipaa-security-and-compliance-on-aws.pdf',
    'https://docs.aws.amazon.com/whitepapers/latest/containers-on-aws/containers-on-aws.pdf'
]

filenames = [
    'AWS-security-whitepaper.pdf',
    'AWS-containers-whitepaper.pdf'
]

metadata = [
    dict(year=2023, source=filenames[0]),
    dict(year=2023, source=filenames[1])
]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

After downloading we can load the documents with the help of `DirectoryLoader` from `PyPDF` available under LangChain and splitting them into smaller chunks.

Note: For the sake of this use-case we are creating chunks of roughly 4000 characters with an overlap of 100 characters using `RecursiveCharacterTextSplitter`.

#### HIPAA Compliance document

In this section, we will load the HIPAA compliance document with `PyPDFLoader`, append document fragments with the metadata, and use LangChain's `RecursiveCharacterTextSplitter` to split the documents in `hipaa_documents` list into smaller text chunks using the `split_documents` method. 

In [9]:
#document 1 (HIPAA COMPLIANCE ON AWS)
hipaa_documents = []

# Load only the first file
hipaa_file = filenames[0]
hipaa_loader = PyPDFLoader(data_root + hipaa_file)
hipaa_document = hipaa_loader.load()

for idx, hipaa_document_fragment in enumerate(hipaa_document):
    hipaa_document_fragment.metadata = metadata[0] if metadata else {}
    hipaa_documents.append(hipaa_document_fragment)
    
#chunking
hipaa_doc_text_splitter = RecursiveCharacterTextSplitter(
    # Set a  small chunk size, just to show.
    chunk_size=2000,
    chunk_overlap=100,
)

hipaa_docs = hipaa_doc_text_splitter.split_documents(hipaa_documents)
print(hipaa_docs[0])

#chunked doc count
hipaa_chunked_count = len(hipaa_docs)
print(
    f"\nNumber of documents chunked and created from the HIPAA Security document: {hipaa_chunked_count}"
)

page_content='AWS Whitepaper\nArchitecting for HIPAA Security and \nCompliance on Amazon Web Services\nCopyright © 2024 Amazon Web Services, Inc. and/or its aﬃliates. All rights reserved.' metadata={'year': 2023, 'source': 'AWS-security-whitepaper.pdf'}

Number of documents chunked and created from the HIPAA Security document: 152


#### Containers on AWS Document

In this section, we will load the Containers on AWS document with `PyPDFLoader`, append document fragments with the metadata, and use LangChain's `RecursiveCharacterTextSplitter` to split the documents in `container_documents` list into smaller text chunks using the `split_documents` method. 

In [10]:
#document 2 (Containers on AWS)
container_documents = []

# Load only the second file
container_file = filenames[1]
container_loader = PyPDFLoader(data_root + container_file)
container_document = container_loader.load()

for idx, container_document_fragment in enumerate(container_document):
    container_document_fragment.metadata = metadata[0] if metadata else {}
    container_documents.append(container_document_fragment)
    
#chunking
container_text_splitter = RecursiveCharacterTextSplitter(
    # Set a small chunk size, just to show.
    chunk_size=2000,
    chunk_overlap=100,
)

container_docs = container_text_splitter.split_documents(container_documents)
print(container_docs[1])

#chunked doc count
container_chunked_count = len(container_docs)
print(
    f"\nNumber of documents chunked and created from the original: {container_chunked_count}"
)

page_content="Containers on AWS AWS Whitepaper\nContainers on AWS: AWS Whitepaper\nCopyright © 2024 Amazon Web Services, Inc. and/or its aﬃliates. All rights reserved.\nAmazon's trademarks and trade dress may not be used in connection with any product or service \nthat is not Amazon's, in any manner that is likely to cause confusion among customers, or in any \nmanner that disparages or discredits Amazon. All other trademarks not owned by Amazon are \nthe property of their respective owners, who may or may not be aﬃliated with, connected to, or \nsponsored by Amazon." metadata={'year': 2023, 'source': 'AWS-security-whitepaper.pdf'}

Number of documents chunked and created from the original: 57


---

## Summarizing Long Documents with LangChain

In the following sections, we will go over three different summarization techniques with LangChain:
    
 #####   1. Stuff
 #####   2. Map Reduce
 #####   3. Refine
 ---

### 1. Stuff with load_summarize_chain

Stuffing is the simplest method to pass data to a language model. It "stuffs" text into the prompt as context in a way that all of the relevant information can be processed by the model to get what you want.

In LangChain, you can use `StuffDocumentsChain` as part of the `load_summarize_chain` method. What you need to do is set `stuff` as the `chain_type` of your chain.

In [11]:
stuff_summary_chain = load_summarize_chain(llm=llm, chain_type="stuff", verbose=False)

Next, let's take a look at the Prompt template used by the Stuff summarize chain:

In [12]:
stuff_summary_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

Here, we see that by default, the Prompt template for `llm_chain` has been set to: 'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

This can be altered by instantiating using `from_template` with LangChain to set a new prompt. We can do that below:



In [13]:
stuff_prompt = PromptTemplate.from_template('Write a detailed and complete summary of the following:\n\n\n"{text}"\n\n\nDETAILED SUMMARY:')

In [14]:
stuff_summary_chain.llm_chain.prompt.template = stuff_prompt.template #set new prompt template

Now that we have set the new prompt template, let us first try generating a summary of the **Containers on AWS** whitepaper.

In [15]:
try:
    stuff_container_summary = stuff_summary_chain.run(container_docs) 
except Exception as e:
    print(e)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [16]:
print(stuff_container_summary.strip())

The AWS Whitepaper "Containers on AWS" provides guidance and options for running containers on AWS. Containers offer a way to develop, ship, and run applications in an isolated environment. AWS complements containers and offers a wide range of scalable orchestration and infrastructure services for container deployment.

The whitepaper covers various container orchestration and compute options such as AWS App Runner, Amazon Elastic Container Service (Amazon ECS), Amazon Elastic Kubernetes Service (Amazon EKS), and AWS Fargate. It also discusses key considerations for container workloads on AWS, including container runtime, container-enabled AMIs, compute options, scheduling, container repositories, observability, storage, networking, security, build and deploy automation, infrastructure as code and platform automation, and scaling.

The benefits of using containers include speed, consistency, density and resource efficiency, and portability. AWS provides a range of purpose-driven servic

From the cell ouput above, we can see that since Stuffing only requires a single call to the LLM, it can be faster than other methods that require multiple calls. When summarizing text, the model has access to all the data at once, which can result in a fast response for the summary.

Next, let us use the **HIPAA and Security Compliance** on AWS whitepaper to see how the model deals with summarization using the `StuffDocumentChain` when presented with a longer document.

In [17]:
try:
    stuff_hipaa_summary = stuff_summary_chain.run(hipaa_docs) # (ValidationException error) prompt over 32k window length / number of tokens exceeds window size 
except Exception as e:
    print(e)

Error raised by bedrock service: An error occurred (ValidationException) when calling the InvokeModel operation: This model's maximum context length is 32768 tokens. Please reduce the length of the prompt


#### Notes:
In the output for the above cell, we see that an error is raised due to the prompt far exceeding the model's maximum context length. Since stuffing summarizes text by feeding the entire document to a large language model (LLM) in a single call, it is difficult to process long documents. The Mistral models have a context length of 32k tokens, which is the maximum number of tokens that can be processed in a single call. If the document is longer than the context length, stuffing will not work. Also the stuffing method is not suitable for summarizing large documents, as it can be slow and may not produce a good summary.

Let's explore a couple chunk-wise summarization techniques with [LangChain](https://python.langchain.com/docs/get_started/introduction.html) to be able to mitigate the restrictions of your large documents not fitting into the context window of the model.

---

### 2. Map Reduce with load_summarize_chain

The `Map_Reduce` method involves summarizing each document individually (map step) and then combining these summaries into a final summary (reduce step). This approach is more scalable and can handle larger volumes of text. The map reduce technique is designed for summarizing large documents that exceed the token limit of the language model. It involves dividing the document into chunks, generating summaries for each chunk, and then combining these summaries to create a final summary. This method is efficient for handling large files and significantly reduces processing time.

In LangChain, you can use `MapReduceDocumentsChain` as part of the `load_summarize_chain method`. What you need to do is set `map_reduce` as the `chain_type` of your chain.

---

![map-reduce](imgs/mapreduce.png)

In [18]:
# Takes a list of documents, combines them into a single string, and passes this to an LLMChain, it then combines and iteratively reduces the mapped document
map_reduce_summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

The `ReduceDocumentsChain` handles taking the document mapping results and reducing them into a single output. It wraps a generic `CombineDocumentsChain` (like `StuffDocumentsChain`) but adds the ability to collapse documents before passing it to the `CombineDocumentsChain` if their cumulative size exceeds token_max.

In [19]:
# Instantiation using from_template (recommended)
#sets the prompt template for the summaries generated for all the individual document chunks.
initial_map_prompt = PromptTemplate.from_template("""
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """)

map_reduce_summary_chain.llm_chain.prompt.template = initial_map_prompt.template

#sets the prompt template for generating a cumulative summary of all the document chunks for reduce documents chain.
reduce_documents_prompt= PromptTemplate.from_template("""
                      Write a detailed summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """)

map_reduce_summary_chain.reduce_documents_chain.combine_documents_chain.llm_chain.prompt.template = reduce_documents_prompt.template

Here, we perform summarization on the **HIPAA and Security Compliance** document with `Map-Reduce`. Since this is document is quite large, it can take a while to run.
In order to see how Map_Reduce works, let us generate a summary of a subset of the document chunks **(50 to 70)**.

In [20]:
#this cell might take 5-10 minutes to run
try:
    map_reduce_summary = map_reduce_summary_chain.run(hipaa_docs[50:71])  
except Exception as e:
    print(e)

Token indices sequence length is longer than the specified maximum sequence length for this model (5627 > 1024). Running this sequence through the model will result in indexing errors


In [21]:
print(map_reduce_summary.strip())

- The text discusses the requirements for HIPAA (Health Insurance Portability and Accountability Act) security and compliance on Amazon Web Services (AWS).
- To comply with the requirement that Protected Health Information (PHI) be encrypted at rest, Amazon Elastic File System (EFS) supports two methods: selecting the "Enable encryption of data at rest" option when creating a new file system, or encrypting data before placing it on EFS.
- PHI should not be used as all or part of any file name or folder name.
- Encryption of PHI while in transit for Amazon EFS is provided by Transport Layer Security (TLS) between the EFS service and the instance mounting the file system.
- Amazon Elastic Kubernetes Service (Amazon EKS) is a managed service that makes it easy for customers to run Kubernetes on AWS without needing to stand up or maintain their own Kubernetes control plane.
- Amazon ElastiCache for Redis is an in-memory data structure service that can be used as a data store or cache, and 

#### Notes:
With `Map_Reduce`, the model is able to summarize a large document by overcoming the context limit of Stuffing method with parallel processing. 
However, it requires multiple calls to the model and potentially loses context between individual summaries of the chunks. To deal with this challenge, let us try another method that performs chunk-wise summarization.

---

### 3. Refine with load_summarize_chain

The `Refine` method iteratively updates its answer by looping over the input documents. For each document, it passes all non-document inputs, the current document, and the latest intermediate answer to an LLM chain to get a new answer. This method is useful for refining a summary based on new context.`Refine` is a simpler alternative to `Map_Reduce` technique. It involves generating a summary for the first chunk, combining it with the second chunk, generating another summary, and continuing this process until a final summary is achieved. This method is suitable for large documents but requires less complexity compared to `Map_Reduce`.

---

![map-reduce](imgs/refine.png)



In [22]:
# Run an initial prompt on a small chunk of data to generate a summary. Then, for each subsequent document, the output from the previous document is passed in along with the new document, and the LLM is asked to refine the output based on the new document.
refine_summary_chain = load_summarize_chain(llm=llm, chain_type="refine", verbose=False)
refine_summary_chain_french = load_summarize_chain(llm=llm, chain_type="refine", verbose=False) #refine summary chain for summarization in french

Here, we perform summarization on the **HIPAA and Security Compliance** document with `Refine`. Since this is document is quite large, it can take a while to run.
In order to see how Refine works, let us generate a summary of a subset of the document chunks **(50 to 70)**.

In [23]:
#initial llm chain prompt template
initial_refine_prompt = PromptTemplate.from_template("""
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """)

refine_summary_chain.initial_llm_chain.prompt.template = initial_refine_prompt.template

#refine llm chain prompt template
refine_documents_prompt= PromptTemplate.from_template("Your job is to produce a final summary.\nWe have provided an existing summary up to a certain point: {existing_answer}\nWe have the opportunity to refine the existing summary (only if needed) with some more context below.\n------------\n{text}\n------------\nGiven the new context, refine the original summary.\nIf the context isn't useful, return the original summary.")

refine_summary_chain.refine_llm_chain.prompt.template = refine_documents_prompt.template

In [24]:
try:
    refine_summary = refine_summary_chain.run(hipaa_docs[50:71])
except Exception as e:
    print(e)

In [25]:
print(refine_summary.strip())

The text discusses the measures to ensure HIPAA security and compliance on Amazon Web Services (AWS), focusing on various services including Amazon Elastic File System (EFS), Amazon Elastic Kubernetes Service (Amazon EKS), Amazon ElastiCache for Redis, Amazon OpenSearch Service, Amazon EMR, Amazon EventBridge, Amazon Forecast, Amazon FSx, Amazon HealthLake, and Amazon Inspector.

For EFS and Amazon FSx, the text outlines two methods to encrypt Protected Health Information (PHI) at rest. The first method involves enabling encryption during the creation of a new file system, which ensures all data is encrypted using AES-256 encryption and AWS Key Management Service (KMS)-managed keys. The second method is to encrypt data before placing it on EFS or FSx, but this places the responsibility of managing the encryption process and key management on the customer. The text also advises against using PHI as part of any file or folder name. For PHI in transit, encryption is provided by Transport 

---
Now that we have seen how the `refine` document chain constructs a response, let us try altering the refine_llm_chain prompt template to help highlight some of the multilingual capabilties of the [Mistral Large](https://mistral.ai/news/mistral-large/) model. Mistral Large demonstrates superior capabilities in handling multi-lingual tasks. Mistral-large has been specifically trained to understand and generate text in multiple languages, especially in French, German, Spanish, and Italian. This can be especially valuable for businesses and users that need to communicate in multiple languages. In the below cell, we set the `refine llm chain` prompt template to return the final summary in French.

In [231]:
#refine llm chain prompt template
refine_documents_prompt_french= PromptTemplate.from_template("Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in French"
    "If the context isn't useful, return the original summary.")

refine_summary_chain_french.refine_llm_chain.prompt.template = refine_documents_prompt_french.template

In [233]:
try:
    refine_summary_french = refine_summary_chain_french.run(hipaa_docs[50:71])
except Exception as e:
    print(e)

In [235]:
print(refine_summary_french.strip())

Le texte traite des mesures visant à garantir la sécurité et la conformité HIPAA sur Amazon Web Services (AWS), en se concentrant sur divers services tels qu'Amazon Elastic File System (EFS), Amazon Elastic Kubernetes Service (Amazon EKS), Amazon ElastiCache for Redis, Amazon OpenSearch Service, Amazon EventBridge, Amazon Forecast, Amazon FSx, Amazon GuardDuty, Amazon HealthLake, Amazon Inspector, Amazon Managed Service for Apache Flink, Amazon Kinesis Streams, Amazon Data Firehose, Amazon Kinesis Video Streams, Amazon Managed Streaming for Apache Kafka (Amazon MSK), Amazon MQ et Amazon Lex. Il souligne l'importance de surveiller et de journaliser toutes les activités liées aux PHI sur AWS à l'aide d'outils tels qu'AWS CloudTrail et Amazon CloudWatch.

Pour chaque service, le texte insiste sur les méthodes de chiffrement des PHI au repos et en transit, ainsi que sur les meilleures pratiques pour garantir la conformité HIPAA. Le texte mentionne également l'utilisation d'Amazon Neptune, 

### Notes:
`Refine` has the potential to incorporate more relevant context compared to `Map_Reduce`, potentially resulting in a more comprehensive and accurate summary. However, it comes with a trade-off: `Refine` necessitates a significantly higher number of calls to the LLM than the `Stuff` and `Map_Reduce` since it is an incremental process where the subsequent chunk's summary uses the previous chunk's summary. Moreover, these calls are not independent, which means they cannot be parallelized, potentially leading to longer processing times. Another consideration is that the Refine method may exhibit recency bias, where the most recent documents in the sequence could carry more weight or influence in the final summary, as the method processes documents in a specific order.

---
## Distributors
- Amazon Web Services
- Mistral AI

